# Go through the cells until you see STOP in markdown

In [1]:
from pyspark.sql import SparkSession
from h2o.estimators import *
from pysparkling import *
import h2o
import shutil
from tqdm.notebook import tqdm
spark = SparkSession.builder.config('spark.dynamicAllocation.enabled','false').getOrCreate()

## Create H2O Sparkling Water cluster

In [2]:
conf = H2OConf().setInternalClusterMode()
hc = H2OContext.getOrCreate(conf)

Connecting to H2O server at http://10.129.34.72:54321 ... successful.


H2O_cluster_uptime:,14 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,4 months and 3 days !!!
H2O_cluster_name:,sparkling-water-jovyan_spark-application-1622826988926
H2O_cluster_total_nodes:,2
H2O_cluster_free_memory:,7.667 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.4-1-3.0
 * H2O name: jovyan
 * cluster size: 2
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,10.129.34.38,54321)
  (1,10.129.35.38,54321)
  ------------------------

  Open H2O Flow in browser: http://10.129.34.72:54321 (CMD + click in Mac OSX)

    


In [3]:
from splicemachine.spark.context import PySpliceContext
splice = PySpliceContext(spark)
from splicemachine.mlflow_support import *
from splicemachine.mlflow_support.utilities import get_user
mlflow.register_splice_context(splice)
mlflow.set_experiment('h2o deployment')
schema = get_user()

INFO: 'h2o deployment' does not exist. Creating a new experiment


## GBM

In [4]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator


# Import the titanic dataset into H2O:
titanic = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")

# Set the predictors and response; set the response as a factor:
titanic["survived"] = titanic["survived"].asfactor()
predictors = ['sibsp', 'sex', 'age']
response = "survived"

# Split the dataset into a train and valid set:
train, valid = titanic.split_frame(ratios=[.8], seed=1234)


model = H2OGradientBoostingEstimator(ntrees         =50,
                                        max_depth      =6,
                                        learn_rate     =0.1, 
                                        nfolds         =2)

model.train(x               =predictors,
               y               =response,
               training_frame  =train,
               validation_frame=valid
               )


print('deploying gbm')
splice.dropTableIfExists(f'{schema}.h2o_gbm')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_gbm',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)

print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)
print('Running model')
splice.execute("insert into h2o_gbm (moment_key,sibsp,sex,age) values(0, 5,'female',2.496)")    
splice.execute("insert into h2o_gbm (moment_key,sibsp,sex,age) values(1, 1,'male',-7.048)")
print('Getting results')
data = splice.df('select sibsp, sex, age, prediction, c0, c1 from h2o_gbm').toPandas()
print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['SIBSP','SEX','AGE']])
db_preds = data[['PREDICTION','C0','C1']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)



for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    db_p, db_c0, db_c1 = db_pred[1]
    raw_p, raw_c0, raw_c1 = raw_pred[1]
    # Table returns string but h2o returns int index
    raw_p = f'C{int(raw_p)}'
    
    assert db_p == raw_p, f'Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}'
    assert round(db_c0,14) == round(raw_c0,14), f'Something is wrong. Model Table gives {db_c0} but raw model gives {raw_c0}'
    assert round(db_c1,14) == round(raw_c1,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deploying gbm
Uploading file... Done.
Deploying model to database...


Passing in primary keys as a list of tuples is deprecated. Use dictionary {column name: type}


Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:17:21.788 - A service worker has found your request
INFO     2021-06-04 17:17:21.925 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:17:22.064 - Handler is available
INFO     2021-06-04 17:17:22.088 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:17:22.196 - Retrieved MLFlow Run
INFO     2021-06-04 17:17:22.227 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:17:22.350 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 17:17:22.384 - Extracting Model from DB with Name: model
INFO     2021-06-04 17:17:22.429 - Decoding Model Artifact Binary Stream for Deployment
INFO     2021-06-04 17:17:22.475 - Decompressing Model Ar

## Multinomial Model Example

In [5]:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
cars["cylinders"] = cars["cylinders"].asfactor()
r = cars[0].runif()
train = cars[r > .2]
valid = cars[r <= .2]
response_col = "cylinders"
distribution = "multinomial"
predictors = ["displacement","power","weight","acceleration","year_make"]

train.rename(columns={'year':'year_make'})

model = H2OGradientBoostingEstimator(nfolds=3,
                                   distribution=distribution)
model.train(x=predictors,
          y=response_col,
          training_frame=train,
          validation_frame=valid)



print('deploying multinomial')
splice.dropTableIfExists(f'{schema}.h2o_multinomial')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_multinomial',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)
print('Running model')
splice.execute("insert into h2o_multinomial (displacement,power,weight,acceleration,year_make, moment_key) values(18,101,22,23.142,1,1)")
splice.execute("insert into h2o_multinomial (displacement,power,weight,acceleration,year_make, moment_key) values(18,6,232,100,3,2)")
print('Getting results')
data = splice.df("select displacement,power,weight,acceleration,year_make,prediction, c3,c4,c5,c6,c8 from h2o_multinomial").toPandas()
print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['DISPLACEMENT','POWER','WEIGHT','ACCELERATION','YEAR_MAKE']])
db_preds = data[['PREDICTION','C3','C4','C5','C6','C8']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    db_p, db_c0, db_c1, db_c2, db_c3, db_c4 = db_pred[1]
    raw_p, raw_c0, raw_c1, raw_c2, raw_c3, raw_c4 = raw_pred[1]
    # Table returns string but h2o returns int index
    raw_p = f'C{int(raw_p)}'
    
    assert db_p == raw_p, f'Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}'
    assert round(db_c0,14) == round(raw_c0,14), f'Something is wrong. Model Table gives {db_c0} but raw model gives {raw_c0}'
    assert round(db_c1,14) == round(raw_c1,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'
    assert round(db_c2,14) == round(raw_c2,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'
    assert round(db_c3,14) == round(raw_c3,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'
    assert round(db_c4,14) == round(raw_c4,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deploying multinomial
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:19:45.741 - A service worker has found your request
INFO     2021-06-04 17:19:45.801 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:19:45.861 - Handler is available
INFO     2021-06-04 17:19:45.883 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:19:45.996 - Retrieved MLFlow Run
INFO     2021-06-04 17:19:46.025 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:19:46.098 - Reading Model Artifact Stream fro

## Ordinal Model Example

In [6]:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
cars["cylinders"] = cars["cylinders"].asfactor()
cars.rename(columns={'year':'year_make'})
r = cars[0].runif()
train = cars[r > .2]
valid = cars[r <= .2]
response = "cylinders"
predictors = ["displacement","power","weight","acceleration","year_make"]
model = H2OGeneralizedLinearEstimator(seed=1234,
                                         family='ordinal')
model.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid)


print('deploying ordinal')
splice.dropTableIfExists(f'{schema}.h2o_ordinal')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_ordinal',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)
print('Running model')
splice.execute("insert into h2o_ordinal (displacement,power,weight,acceleration,year_make, moment_key) values(18,101,22,23.142,1,1 )")
splice.execute("insert into h2o_ordinal (displacement,power,weight,acceleration,year_make, moment_key) values(18,6,232,100,3,2)")
print('Getting results')
data = splice.df("select displacement,power,weight,acceleration,year_make,prediction, c3,c4,c5,c6,c8 from h2o_ordinal").toPandas()
print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['DISPLACEMENT','POWER','WEIGHT','ACCELERATION','YEAR_MAKE']])
db_preds = data[['PREDICTION','C3','C4','C5','C6','C8']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    db_p, db_c0, db_c1, db_c2, db_c3, db_c4 = db_pred[1]
    raw_p, raw_c0, raw_c1, raw_c2, raw_c3, raw_c4 = raw_pred[1]
    # Table returns string but h2o returns int index
    raw_p = f'C{int(raw_p)}'
    
    assert db_p == raw_p, f'Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}'
    assert round(db_c0,14) == round(raw_c0,14), f'Something is wrong. Model Table gives {db_c0} but raw model gives {raw_c0}'
    assert round(db_c1,14) == round(raw_c1,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'
    assert round(db_c2,14) == round(raw_c2,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'
    assert round(db_c3,14) == round(raw_c3,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'
    assert round(db_c4,14) == round(raw_c4,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%


Reached maximum number of iterations 50!


deploying ordinal
Uploading file... Done.
Deploying model to database...


Passing in primary keys as a list of tuples is deprecated. Use dictionary {column name: type}


Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:21:55.037 - A service worker has found your request
INFO     2021-06-04 17:21:55.089 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:21:55.141 - Handler is available
INFO     2021-06-04 17:21:55.160 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:21:55.260 - Retrieved MLFlow Run
INFO     2021-06-04 17:21:55.288 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:21:55.361 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 17:21:55.390 - Extracting Model from DB with Name: model
INFO     2021-06-04 17:21:55.428 - Decoding Model Artifact Binary Stream for Deployment
INFO     2021-06-04 17:21:55.463 - Decompressing Model Ar

## Regression (Gradient Boosting Estimator) Model Example

In [8]:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
cars.rename(columns={'year':'year_make'})
r = cars[0].runif()
train = cars[r > .2]
valid = cars[r <= .2]
response_col = "economy"
distribution = "gaussian"
predictors = ["displacement","power","weight","acceleration","year_make"]
model = H2OGradientBoostingEstimator(nfolds=3,
                                   distribution=distribution,
                                   fold_assignment="Random")
model.train(x=predictors,
          y=response_col,
          training_frame=train,
          validation_frame=valid)
model.plot(timestep="AUTO", metric="AUTO",)


print('deploying regression')
splice.dropTableIfExists(f'{schema}.h2o_regression')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_regression',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)
print('Running model')
splice.execute("insert into h2o_regression (displacement,power,weight,acceleration,year_make, moment_key) values(18,101,22,23.142,1,1 )")
splice.execute("insert into h2o_regression (displacement,power,weight,acceleration,year_make, moment_key) values(18,6,232,100,3,2)")
print('Getting results')
data = splice.df("select displacement,power,weight,acceleration,year_make,prediction from h2o_regression").toPandas()
print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['DISPLACEMENT','POWER','WEIGHT','ACCELERATION','YEAR_MAKE']])
db_preds = data[['PREDICTION']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    db_p = db_pred[1]['PREDICTION']
    raw_p = raw_pred[1]['predict']
    
    l = min(16, len(str(db_p)), len(str(raw_p))) -2 # -2 because of the decimal point and final value
    try:
        assert round(db_p,l) == round(raw_p,l), f'Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}'
    except:
        print(f"Models do not match to {l} decimals. Testing precision")
        while l > 8:
            if round(db_p,l) == round(raw_p,l):
                print(f'Values match to {l} decimals')
                break
            l -= 1
        if l <= 8:
            raise Exception(f"Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}")

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
deploying regression
Table exists. Dropping table
Uploading file... Done.
Deploying model to database...


Passing in primary keys as a list of tuples is deprecated. Use dictionary {column name: type}


Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:26:18.294 - A service worker has found your request
INFO     2021-06-04 17:26:18.396 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:26:18.549 - Handler is available
INFO     2021-06-04 17:26:18.566 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:26:18.783 - Retrieved MLFlow Run
INFO     2021-06-04 17:26:18.808 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:26:18.875 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 17:26:18.902 - Extracting Model from DB with Name: model
INFO     2021-06-04 17:26:18.942 - Decoding Model Artifact Binary Stream for Deployment
INFO     2021-06-04 17:26:18.997 - Decompressing Model Ar

## HGLM Model example

In [9]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
cars.rename(columns={'year':'year_make'})
r = cars[0].runif()
train = cars[r > .2]
valid = cars[r <= .2]
response = "economy"
distribution = "gaussian"
predictors = ["displacement","power","weight","acceleration","year_make"]
model = H2OGeneralizedLinearEstimator(alpha=.25)
model.train(x=predictors,
                 y=response,
                 training_frame=train,
                 validation_frame=valid)


print('deploying hglm')
splice.dropTableIfExists(f'{schema}.h2o_hglm')

while mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_hglm',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)
print('Running model')
splice.execute("insert into h2o_hglm (displacement,power,weight,acceleration,year_make, moment_key) values(18,101,22,23.142,1,1 )")
splice.execute("insert into h2o_hglm (displacement,power,weight,acceleration,year_make, moment_key) values(18,6,232,100,3,2)")
print('Getting results')
data = splice.df("select displacement,power,weight,acceleration,year_make,prediction from h2o_hglm").toPandas()
print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['DISPLACEMENT','POWER','WEIGHT','ACCELERATION','YEAR_MAKE']])
db_preds = data[['PREDICTION']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    db_p = db_pred[1]['PREDICTION']
    raw_p = raw_pred[1]['predict']
    
    l = min(16, len(str(db_p)), len(str(raw_p))) -2    
    try:
        assert round(db_p,l) == round(raw_p,l), f'Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}'
    except:
        print(f"Models do not match to {l} decimals. Testing precision")
        while l > 8:
            if round(db_p,l) == round(raw_p,l):
                print(f'Values match to {l} decimals')
                break
            l -= 1
        if l <= 8:
            raise Exception(f"Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}")

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
deploying hglm
Uploading file... Done.
Deploying model to database...


Passing in primary keys as a list of tuples is deprecated. Use dictionary {column name: type}


Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:27:20.187 - A service worker has found your request
INFO     2021-06-04 17:27:20.239 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:27:20.289 - Handler is available
INFO     2021-06-04 17:27:20.308 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:27:20.409 - Retrieved MLFlow Run
INFO     2021-06-04 17:27:20.436 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:27:20.522 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 17:27:20.545 - Extracting Model from DB with Name: model
INFO     2021-06-04 17:27:20.580 - Decoding Model Artifact Binary Stream for Deployment
INFO     2021-06-04 17:27:20.609 - Decompressing Model Ar

## Word2Vec example

In [10]:
from h2o.estimators.word2vec import H2OWord2vecEstimator
job_titles_path = "https://raw.githubusercontent.com/h2oai/sparkling-water/rel-1.6/examples/smalldata/craigslistJobTitles.csv"
job_titles = h2o.import_file(job_titles_path, destination_frame = "jobtitles",
                             col_names = ["category", "jobtitle"], col_types = ["enum", "string"], header = 1)
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words


words = tokenize(job_titles["jobtitle"])
words.columns = ['word']

print("Build word2vec model")
model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10, vec_size=15)
model.train(training_frame=words)


print("Sanity check - find synonyms for the word 'teacher'")
model.find_synonyms("teacher", count = 5)

print('deploying w2v')
splice.dropTableIfExists(f'{schema}.h2o_w2v')

while mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_w2v',mlflow.current_run_id(), df=hc.asSparkFrame(words), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)
print('Running model')

splice.execute("insert into h2o_w2v (word, moment_key) values('teacher', 1)")
splice.execute("insert into h2o_w2v (word, moment_key) values('teachers', 2)")
splice.execute("insert into h2o_w2v (word, moment_key) values('elementary', 3)")
print('Getting results')
data = splice.df("select * from h2o_w2v").toPandas()
print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['WORD']])
db_preds = data[['WORD_C0','WORD_C1','WORD_C2','WORD_C3','WORD_C4','WORD_C5','WORD_C6','WORD_C7','WORD_C8','WORD_C9','WORD_C10','WORD_C11','WORD_C12','WORD_C13','WORD_C14']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.transform(features,aggregate_method=None).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
#     db_p, db_c0, db_c1, db_c2, db_c3, db_c4 = db_pred[1]
#     raw_p, raw_c0, raw_c1, raw_c2, raw_c3, raw_c4 = raw_pred[1]
    # Table returns string but h2o returns int index
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,14) == round(raw,14), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Build word2vec model
word2vec Model Build progress: |██████████████████████████████████████████| 100%
Sanity check - find synonyms for the word 'teacher'
deploying w2v
Uploading file... Done.
Deploying model to database...


Passing in primary keys as a list of tuples is deprecated. Use dictionary {column name: type}


Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:28:51.422 - A service worker has found your request
INFO     2021-06-04 17:28:51.509 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:28:51.599 - Handler is available
INFO     2021-06-04 17:28:51.614 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:28:51.721 - Retrieved MLFlow Run
INFO     2021-06-04 17:28:51.744 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:28:51.847 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 17:28:51.873 - Extracting Model from DB with Name: model
INFO     2021-06-04 17:28:51.901 - Decoding Model Artifact Binary Stream for Deployment
INFO     2021-06-04 17:28:51.934 - Decompressing Model Ar

## AutoEncoder Example

In [11]:
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator
# train = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/bigdata/laptop/mnist/train.csv.gz")
# test = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/bigdata/laptop/mnist/test.csv.gz")

import random
train = []
test = []
for i in range(60000):
    train.append([random.random()*4 for i in range(6)])
    test.append([random.random()*4 for i in range(6)])

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)
predictors = [f'C{i}' for i in range(1,7)]


model = H2OAutoEncoderEstimator(activation="Tanh",
                                   hidden=[2],
                                   l1=1e-5,
                                   ignore_const_cols=False,
                                   epochs=1)
model.train(x=predictors,training_frame=train)
test_rec_error = model.anomaly(test)
test_rec_error
test_rec_error_features = model.anomaly(test, per_feature=True)
print(test_rec_error_features)
model.predict(test)

print('deploying autoencoder')
splice.dropTableIfExists(f'{schema}.h2o_ae')

while mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_ae',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)

print('Running model')
splice.execute("insert into h2o_ae (moment_key,c1,c2,c3,c4,c5,c6) values(1,0, 0, 0, 0, 0, 0)")
splice.execute("insert into h2o_ae (moment_key,c1,c2,c3,c4,c5,c6) values(2,1, 1, 1, 1, 1, 1)")
splice.execute("insert into h2o_ae (moment_key,c1,c2,c3,c4,c5,c6) values(3,0.25, 0.99, 0.623, 0.21, 0.52, 0.66)")

print('Getting results')
data = splice.df("select * from h2o_ae").toPandas()

print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['C1','C2','C3','C4','C5','C6']])
db_preds = data[['C1_RECONSTR','C2_RECONSTR','C3_RECONSTR','C4_RECONSTR','C5_RECONSTR','C6_RECONSTR']]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

# Check the MSE score
raw_mses = model.anomaly(features).as_data_frame(use_pandas=True)
db_mses = data[['MSE_RECONSTR']]
for db_mse, raw_mse in zip(db_mses.iterrows(), raw_mses.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'
    
print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%


reconstr_C1.SE,reconstr_C2.SE,reconstr_C3.SE,reconstr_C4.SE,reconstr_C5.SE,reconstr_C6.SE
0.0593547,0.00875459,0.0477355,0.0933169,0.00361706,3.04805e-06
0.131604,0.0485755,0.101616,0.0259252,0.0259555,0.0575199
0.00913057,8.38834e-06,0.177314,0.0649745,0.0945497,0.0310709
0.0599686,0.00177825,0.00718096,0.270593,0.00650925,0.00312805
0.0179738,0.00337873,0.0634103,0.102517,0.0947999,0.0236578
0.0422885,6.55559e-05,0.322466,0.00214568,0.0302279,0.0119136
0.0187796,0.0109573,0.00151358,0.0633876,0.0867147,0.00207403
0.00595111,0.0299952,0.00264518,0.0835673,0.104071,0.00372481
0.00092647,0.00302635,0.0556849,0.0752193,0.022254,0.00675892
0.00114563,0.00165328,0.0992462,0.0444607,0.000335413,0.00705533



deeplearning prediction progress: |███████████████████████████████████████| 100%
deploying autoencoder
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:31:49.765 - A service worker has found your request
INFO     2021-06-04 17:31:49.811 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:31:49.859 - Handler is available
INFO     2021-06-04 17:31:49.873 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:31:49.993 - Retrieved MLFlow Run
INFO     2021-06-04 17:31:50.015 - Updating MLFlow Run for the UI
INFO     2021-06-04 17:31:50.078 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 17:31:50.104 - Extracting Model from DB wit

## Clustering

In [12]:
from h2o.estimators.kmeans import H2OKMeansEstimator
import random

train = []
test = []
for i in range(60000):
    train.append([random.random()*4 for i in range(6)])
    test.append([random.random()*4 for i in range(6)])

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)
predictors = [f'C{i}' for i in range(1,7)]


model = H2OKMeansEstimator(k=3, nfolds=3)
model.train(x=list(range(4)), training_frame=train)


print('deploying clustering')
splice.dropTableIfExists(f'{schema}.h2o_cluster')

while mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_cluster',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)

print('Running model')
splice.execute("insert into h2o_cluster (moment_key,c1,c2,c3,c4,c5,c6) values(1,0, 0, 0, 0, 0, 0)")
splice.execute("insert into h2o_cluster (moment_key,c1,c2,c3,c4,c5,c6) values(2,0.258682,2.03961,3.13087,2.71747,2.46077,0.24339)")
splice.execute("insert into h2o_cluster (moment_key,c1,c2,c3,c4,c5,c6) values(3,0.25, 0.99, 0.623, 0.21, 0.52, 0.66)")

print('Getting results')
data = splice.df("select * from h2o_cluster").toPandas()

print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['C1','C2','C3','C4','C5','C6']])
db_preds = data[['PREDICTION']]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

    
print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
deploying clustering
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 17:33:43.067 - A service worker has found your request
INFO     2021-06-04 17:33:43.110 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 17:33:43.158 - Handler is available
INFO     2021-06-04 17:33:43.175 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 17:33:43.279 - Retrieved MLFlow Run
INFO     2021-06-04 17:33:43.298 - Updating MLFlow Run

## PCA Example

In [13]:
from h2o.estimators import H2OPrincipalComponentAnalysisEstimator

train = []
test = []
for i in range(60000):
    train.append([random.random()*4 for i in range(6)])
    test.append([random.random()*4 for i in range(6)])

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)
predictors = [f'C{i}' for i in range(1,7)]

model = H2OPrincipalComponentAnalysisEstimator(k = 4, transform = "STANDARDIZE", pca_method="Power",
                   use_all_factor_levels=True, impute_missing=True)
model.train(x=train.names, training_frame=train)


print('deploying pca')
splice.dropTableIfExists(f'{schema}.h2o_pca')

while mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_pca',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)


print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)

print('Running model')
splice.execute("insert into h2o_pca (moment_key,c1,c2,c3,c4,c5,c6) values(1,0, 0, 0, 0, 0, 0)")
splice.execute("insert into h2o_pca (moment_key,c1,c2,c3,c4,c5,c6) values(2,0.258682,2.03961,3.13087,2.71747,2.46077,0.24339)")
splice.execute("insert into h2o_pca (moment_key,c1,c2,c3,c4,c5,c6) values(3,0.25, 0.99, 0.623, 0.21, 0.52, 0.66)")

print('Getting results')
data = splice.df("select * from h2o_pca").toPandas()

print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['C1','C2','C3','C4','C5','C6']])
db_preds = data[['PC0','PC1','PC2','PC3']]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

    
print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%


_train: PCA Power method failed to converge within TOLERANCE.  Increase max_iterations or reduce TOLERANCE to mitigate this problem.


deploying pca
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 20:38:40.439 - A service worker has found your request
INFO     2021-06-04 20:38:40.504 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 20:38:40.569 - Handler is available
INFO     2021-06-04 20:38:40.585 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 20:38:40.679 - Retrieved MLFlow Run
INFO     2021-06-04 20:38:40.702 - Updating MLFlow Run for the UI
INFO     2021-06-04 20:38:40.763 - Reading Model Artifact Stream from Splice Machine
INFO     2021-06-04 20:38:40.787 - Extracting Model from DB with Name: model
INFO     2021-06-04 20:38:40.819 - Decoding Model Artifact Binary Stream for

## Isolation Forest Example

In [14]:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
cars.rename(columns={'year':'year_make'})
predictors = ["displacement","power","weight","acceleration","year_make"]
model = H2OIsolationForestEstimator(seed=1234,score_each_iteration=True,score_tree_interval=5)
model.train(x=predictors,
              training_frame=cars)
model.model_performance()


print('deploying isoforect')
splice.dropTableIfExists(f'{schema}.h2o_iso')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_iso',mlflow.current_run_id(), df=hc.asSparkFrame(cars).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)
    
print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)

print('Running model')
splice.execute("insert into h2o_iso (displacement,power,weight,acceleration,year_make, moment_key) values(18,101,22,23.142,1,1 )")
splice.execute("insert into h2o_iso (displacement,power,weight,acceleration,year_make, moment_key) values(18,6,232,100,3,2)")

print('Getting results')
data = splice.df("select displacement,power,weight,acceleration,year_make, normalizedScore,score from h2o_iso").toPandas()

print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['DISPLACEMENT','POWER','WEIGHT','ACCELERATION','YEAR_MAKE']])
db_preds = data[['NORMALIZEDSCORE','SCORE']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)

for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
isolationforest Model Build progress: |███████████████████████████████████| 100%
deploying isoforect
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 20:40:13.679 - A service worker has found your request
INFO     2021-06-04 20:40:13.724 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 20:40:13.768 - Handler is available
INFO     2021-06-04 20:40:13.784 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 20:40:13.893 - Retrieved MLFlow Run
INFO     2021-06-04 20:40:13.915 - Updating MLFlow Run for the UI
INFO     2021-06-04 20:40:13.971 - Reading Model Artifact Stream from 

## Neural Network

In [18]:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
cars.rename(columns={'year':'year_make'})
cars["cylinders"] = cars["cylinders"].asfactor()
r = cars[0].runif()
train = cars[r > .2]
valid = cars[r <= .2]
response_col = "cylinders"

predictors = ["displacement","power","weight","acceleration","year_make"]



model = H2ODeepLearningEstimator(variable_importances=True,loss ="Automatic")

model.train(
    x = predictors,
    y = response_col,
    training_frame = train,
    validation_frame = valid
)



print('deploying neural network')
splice.dropTableIfExists(f'{schema}.h2o_nn')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_nn',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)

print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)

print('Running model')
splice.execute("insert into h2o_nn (displacement,power,weight,acceleration,year_make, moment_key) values(18,101,22,23.142,1,1 )")
splice.execute("insert into h2o_nn (displacement,power,weight,acceleration,year_make, moment_key) values(18,6,232,100,3,2)")

print('Getting results')
data = splice.df("select displacement,power,weight,acceleration,year_make,prediction, c3,c4,c5,c6,c8 from h2o_nn").toPandas()

print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['DISPLACEMENT','POWER','WEIGHT','ACCELERATION','YEAR_MAKE']])
db_preds = data[['PREDICTION','C3','C4','C5','C6','C8']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        if isinstance(db,str):
            raw = f'C{int(raw)}'
            assert db==raw, f'Something is wrong. Model Table gives {db} but raw model gives {raw}'
        else:
            assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

    
print('test passed!')

Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning Model Build progress: |██████████████████████████████████████| 100%
deploying neural network
Table exists. Dropping table
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 20:43:40.713 - A service worker has found your request
INFO     2021-06-04 20:43:40.757 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 20:43:40.801 - Handler is available
INFO     2021-06-04 20:43:40.816 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 20:43:40.903 - Retrieved MLFlow Run
INFO     2021-06-04 20:43:40.923 - Updating MLFlow Run for the UI
INFO     2021-06-04 20:43:40.977 - R

## XGBoost

In [19]:
import h2o
from h2o.estimators import H2OXGBoostEstimator

# Import the titanic dataset into H2O:
titanic = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")

# Set the predictors and response; set the response as a factor:
titanic["survived"] = titanic["survived"].asfactor()
predictors = ['sibsp', 'sex', 'age']
response = "survived"

# Split the dataset into a train and valid set:
train, valid = titanic.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
model = H2OXGBoostEstimator(booster='dart',
                                  normalize_type="tree",
                                  seed=1234)
model.train(x=predictors,
                  y=response,
                  training_frame=train,
                  validation_frame=valid)

# Eval performance:
perf = model.model_performance()

# Generate predictions on a test set (if necessary):
pred = model.predict(valid)

print('deploying xgb')
splice.dropTableIfExists('splice.h2o_xgb')

with mlflow.start_run() as run:
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db('splice','h2o_xgb',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key={'MOMENT_KEY':'INTEGER'})
    mlflow.watch_job(jid)

print(f'Loading model {run.info.run_uuid}')
mlflow.load_model(run.info.run_uuid)

print('Running model')
splice.execute("insert into splice.h2o_xgb (moment_key,sibsp,sex,age) values(0, 5,'female',2.496)")
splice.execute("insert into splice.h2o_xgb (moment_key,sibsp,sex,age) values(1, 1,'male',-7.048)")

print('Getting results')
data = splice.df("select sibsp, sex, age, prediction, c0, c1 from splice.h2o_xgb").toPandas()

print('Comparing DB results to model results')
features = h2o.H2OFrame(data[['SIBSP','SEX','AGE']])
db_preds = data[['PREDICTION','C0','C1']]
features.columns = [i.lower() for i in features.columns]

raw_preds = model.predict(features).as_data_frame(use_pandas=True)



for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    db_p, db_c0, db_c1 = db_pred[1]
    raw_p, raw_c0, raw_c1 = raw_pred[1]
    # Table returns string but h2o returns int index
    raw_p = f'C{int(raw_p)}'
    
    assert db_p == raw_p, f'Something is wrong. Model Table gives {db_p} but raw model gives {raw_p}'
    assert round(db_c0,14) == round(raw_c0,14), f'Something is wrong. Model Table gives {db_c0} but raw model gives {raw_c0}'
    assert round(db_c1,14) == round(raw_c1,14), f'Something is wrong. Model Table gives {db_c1} but raw model gives {raw_c1}'

print('test passed!')
    

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost Model Build progress: |███████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
deploying xgb
Uploading file... Done.
Deploying model to database...
Your Job has been submitted. The returned value of this function is the job id, which you can use to monitor the your task in real-time. Run mlflow.watch_job(<job id>) tostream them to stdout, or mlflow.fetch_logs(<job id>) to read them one time to a list

---Job Logs---
INFO     2021-06-04 20:44:35.134 - A service worker has found your request
INFO     2021-06-04 20:44:35.222 - Checking whether handler DEPLOY_DATABASE is enabled
INFO     2021-06-04 20:44:35.306 - Handler is available
INFO     2021-06-04 20:44:35.322 - Retrieving Run from MLFlow Tracking Server...
INFO     2021-06-04 20:44:35.409 - Retrieved MLFlow Run
INFO     2021-06-04 20:44:35.431 - Updating MLFlow Run for th

In [ ]:
spark.stop()

---

# STOP
# ===================================== Below is Broken =====================================

## GLRM (not broken but can't test), TargetEncoder (broken on deploy)

### GLRM explanation https://0xdata.atlassian.net/browse/PUBDEV-7761

## GLRM

In [ ]:
train = []
test = []
for i in range(60000):
    train.append([random.random()*4 for i in range(6)])

train = h2o.H2OFrame(train)
predictors = [f'C{i}' for i in range(1,7)]

model = H2OGeneralizedLowRankEstimator(k=6,seed=1234, impute_original=True,transform='Normalize')
model.train(x=predictors, training_frame=train)

print('deploying glrm')
splice.dropTableIfExists(f'{schema}.h2o_glrm')

while mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run():
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db(schema,'h2o_glrm',mlflow.current_run_id(), df=hc.asSparkFrame(train).select(predictors), create_model_table=True, primary_key=[('MOMENT_KEY','INTEGER')])
    mlflow.watch_job(jid)

In [ ]:

model.reconstruct(h2o.H2OFrame([[0,0,0,0,0,0]]))
# dir(model)

In [ ]:
%%sql
truncate table h2o_glrm;
insert into h2o_glrm (moment_key,c1,c2,c3,c4,c5,c6) values(1,0, 0, 0, 0, 0, 0);
insert into h2o_glrm (moment_key,c1,c2,c3,c4,c5,c6) values(2,0.258682,2.03961,3.13087,2.71747,2.46077,0.24339);
insert into h2o_glrm (moment_key,c1,c2,c3,c4,c5,c6) values(3,0.25, 0.99, 0.623, 0.21, 0.52, 0.66);

select * from h2o_glrm;
select * into ${data_and_preds} from h2o_glrm;

# NOTE
## GLRM Can act as both autoencoders in that they reconstruct data, and also perform PCA. In the database, we always do PCA (maybe something to change later)
## For reconstruction, you call predict. I don't know how to get components

In [ ]:
from beakerx.object import beakerx
data = beakerx.get('data_and_preds')

features = h2o.H2OFrame(data[['C1','C2','C3','C4','C5','C6']])
db_preds = data[['PC0','PC1','PC2','PC3','PC4','PC5']]

# GLRM Can act as both autoencoders in that they reconstruct data, and also perform PCA. In the database, we always do PCA (maybe something to change later)
# To get the PCA, you call proj_archetypes, for reconstruction, you call predict
raw_preds = model.proj_archetypes(features).as_data_frame(use_pandas=True)


for db_pred, raw_pred in zip(db_preds.iterrows(), raw_preds.iterrows()):
    # Check reconstruction values
    for db, raw in zip(db_pred[1], raw_pred[1]):
        assert round(db,13) == round(raw,13), f'Something is wrong. Model Table gives {db} but raw model gives {raw}'

print('test passed!')

### GLRM is weird because it can act as either Dim Reduction or an Autoencoder... Which should we pick?

In [ ]:
pca_path = model.download_mojo('/tmp/gbm.zip')


from py4j.java_gateway import java_import
java_import(splice.jvm, 'hex.genmodel.easy.EasyPredictModelWrapper')
java_import(splice.jvm, 'hex.genmodel.MojoModel')
java_import(splice.jvm, 'java.io.ByteArrayOutputStream')
java_import(splice.jvm, 'java.io.ObjectOutputStream') 
java_import(splice.jvm, 'hex.genmodel.easy.RowData')
java_import(splice.jvm, 'hex.genmodel.easy.prediction.DimReductionModelPrediction')


java_mojo_c = splice.jvm.EasyPredictModelWrapper.Config().setModel(splice.jvm.MojoModel.load(pca_path))
java_mojo = splice.jvm.EasyPredictModelWrapper(java_mojo_c)



m = splice.jvm.MojoModel.load(pca_path)

# java_mojo.predictDimReduction(train)


print(java_mojo.getModelCategory().toString())


row = splice.jvm.RowData()
row.put("C1", "0")
row.put("C2", "0")
row.put("C3", "0")
row.put("C4", "0")
row.put("C5", "0")
row.put("C6", "0")

d = splice.jvm.DimReductionModelPrediction

# pred = m.predictDimReduction(row)

# list(java_mojo.predictDimReduction(row))

In [ ]:
from splicemachine.notebook import hide_toggle
hide_toggle(toggle_next=True)

In [ ]:
%%java
%classpath add jar db-client-3.0.0.1950.jar
%classpath add jar /home/jovyan/nn_model.jar
import java.sql.*; 
import java.util.*; 
import hex.genmodel.easy.RowData;
import hex.genmodel.easy.EasyPredictModelWrapper;
import hex.genmodel.easy.prediction.*;
import java.io.*;
import hex.genmodel.MojoModel;
import hex.genmodel.InMemoryMojoReaderBackend;
import java.sql.Driver;  
import com.splicemachine.db.jdbc.*;


Driver d = new com.splicemachine.db.jdbc.ClientDriver();  
DriverManager.registerDriver(d);
Connection conn = DriverManager.getConnection("jdbc:splice://jdbc-test-aks-dev1.dev.splicemachine-dev.io:1527/splicedb;ssl=basic","splice","admin");
PreparedStatement pstmt = conn.prepareStatement("select \"binary\" from mlmanager.artifacts where RUN_UUID=? and NAME=?");
        pstmt.setString(1, "6ba390856894");
        pstmt.setString(2,"h2omojo");
        ResultSet rs = pstmt.executeQuery();
        EasyPredictModelWrapper model = null;
        if(rs.next()) {
            Blob blobModel = rs.getBlob(1);
            InputStream bis = blobModel.getBinaryStream();
            ObjectInputStream ois = new ObjectInputStream(bis);
            model = (EasyPredictModelWrapper) (ois.readObject());
            ois.close();
        }

        

RowData row = new RowData();

row.put("SDSS_J", "000009.26+151754.5");
row.put("R.A.", "9.08519705868309");
row.put("Dec.", "4.932083187033184");
row.put("z", "7.139249327431729");
row.put("u_mag", "7.139249327431729");
row.put("sig_u", "7.139249327431729");
row.put("g_mag", "7.139249327431729");
row.put("sig_g", "7.139249327431729");
row.put("r_mag", "7.139249327431729");
row.put("sig_r", "7.139249327431729");
row.put("i_mag", "7.139249327431729");
row.put("sig_i", "7.139249327431729");
row.put("z_mag", "7.139249327431729");
row.put("sig_z", "7.139249327431729");
row.put("Radio", "7.139249327431729");
row.put("X-ray", "7.139249327431729");
row.put("J_mag", "7.139249327431729");
row.put("sig_J", "7.139249327431729");
row.put("H_mag", "7.139249327431729");
row.put("sig_H", "7.139249327431729");
row.put("K_mag", "7.139249327431729");
row.put("sig_K", "7.139249327431729");
row.put("M_i", "7.139249327431729");


AbstractPrediction p;
p = model.predictDimReduction(row);

final StringBuilder builder = new StringBuilder();
//for(int i = 0; i < classProbs.length; i++){
//    builder.append(i).append("=").append(classProbs[i]).append(";");
//}
//return builder.substring(0, builder.length() - 1);
//return c;

final double[] dim = ((DimReductionModelPrediction) p).dimensions;
//AutoEncoderModelPrediction


//for(int i = 0; i < dim.length; i++){
//    builder.append("PC" + i).append("=").append(Double.toString(dim[i])).append(";");
//}

return dim;
//return builder.substring(0, builder.length() - 1);

## TargetEncoder (BROKEN on Deploy)

In [ ]:
titanic = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")
titanic.rename(columns={'home.dest':'home_dest'})
predictors = ["home_dest", "cabin", "embarked"]
response = "survived"
titanic["survived"] = titanic["survived"].asfactor()
fold_col = "kfold_column"
titanic[fold_col] = titanic.kfold_column(n_folds=5, seed=1234)
model = H2OTargetEncoderEstimator(k=35,
                                       f=25,
                                       blending=True)
model.train(x=predictors,
                 y=response,
                 training_frame=titanic)


print('deploying target encoder')
splice.dropTableIfExists('splice.te')

with mlflow.start_run():
    mlflow.log_model(model, 'model')
    jid = mlflow.deploy_db('splice','te',mlflow.current_run_id(), df=hc.asSparkFrame(titanic).select(predictors), create_model_table=True, primary_key=[('MOMENT_KEY','INTEGER')])
    mlflow.watch_job(jid)

In [ ]:
model.download_mojo(f'./h2o_model.zip')